In [17]:
import os
import numpy as np
import tensorflow as tf
import random
import cv2 as cv

In [18]:
def list_files(dir_path  = 'dataset',img_ext ='.jpg'):

    files =  []
    discarded = 0
    masked_instance = 0
    for r,d,f in os.walk(dir_path):
        for file in f:
            if file.endswith('.txt'):
                with open(os.path.join(dir_path,file),'r') as fp:
                    lines = fp.readlines()
                    if len(lines) >1:
                        discarded +=1
                        continue

                strip = file.strip('.txt')

                image_path  = os.path.join(r,strip+img_ext)
                if os.path.exists(image_path):
                    if lines[0][0] == '0':
                        masked_instance +=1

                    files.append(strip)

    train_len = int(len(files)*0.7)
    val_len = int(len(files)*0.2)

    random.shuffle(files)



    return files[:train_len],files[train_len:train_len+val_len],files[train_len+val_len:]


In [19]:
training_files, validation_files, test_files = list_files()

In [20]:
input_size = 244

def format_image(img, box):
    height, width = img.shape
    max_size = max(height, width)
    r = max_size / input_size
    new_width = int(width / r)
    new_height = int(height / r)
    new_size = (new_width, new_height)
    resized = cv.resize(img, new_size, interpolation= cv.INTER_LINEAR)
    new_image = np.zeros((input_size, input_size), dtype=np.uint8)
    new_image[0:new_height, 0:new_width] = resized

    x, y, w, h = box[0], box[1], box[2], box[3]
    new_box = [int((x - 0.5*w)* width / r), int((y - 0.5*h) * height / r), int(w*width / r), int(h*height / r)]

    return new_image, new_box

In [21]:
def data_load(files, full_data_path = "dataset", image_ext = ".jpg"):
    X = []
    Y = []

    for file in files:
        img = cv.imread(os.path.join(full_data_path, file + image_ext), cv.IMREAD_GRAYSCALE)

        k = 1

        with open(full_data_path + "/" + file + ".txt", 'r') as fp:
            line = fp.readlines()[0]
            if line[0] == '0':
                k = 0

            box = np.array(line[1:].split(), dtype=float)

        img, box = format_image(img, box)
        img = img.astype(float) / 255.
        box = np.asarray(box, dtype=float) / input_size
        label = np.append(box, k)

        X.append(img)
        Y.append(label)

    X = np.array(X)

    X = np.expand_dims(X, axis=3)

    X = tf.convert_to_tensor(X, dtype=tf.float32)

    Y = tf.convert_to_tensor(Y, dtype=tf.float32)

    result = tf.data.Dataset.from_tensor_slices((X, Y))

    return result

In [22]:
raw_train_ds = data_load(training_files)

In [23]:
raw_validation_ds = data_load(validation_files)

In [24]:
raw_test_ds = data_load(test_files)

In [25]:
CLASSES = 2

def format_instance(image, label):
    return image, (tf.one_hot(int(label[4]), CLASSES), [label[0], label[1], label[2], label[3]])

In [26]:
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.1),
  layers.RandomZoom(0.1),
], name="data_augmentation")

In [27]:
BATCH_SIZE = 32

def tune_training_ds(dataset):
    dataset = dataset.map(format_instance, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(1024, reshuffle_each_iteration=True)
    dataset = dataset.repeat()
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(lambda x, y: (data_augmentation(x, training=True), y),
                          num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

In [28]:
train_ds = tune_training_ds(raw_train_ds)

In [29]:
def tune_validation_ds(dataset):
    dataset = dataset.map(format_instance, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(len(validation_files) // 4)
    dataset = dataset.repeat()
    return dataset

In [30]:
validation_ds = tune_validation_ds(raw_validation_ds)

In [31]:
DROPOUT_FACTOR = 0.5

def build_feature_extractor(inputs):

    x = tf.keras.layers.Conv2D(16, kernel_size=3, activation='relu', input_shape=(input_size, input_size, 1))(inputs)
    x = tf.keras.layers.AveragePooling2D(2,2)(x)

    x = tf.keras.layers.Conv2D(32, kernel_size=3, activation = 'relu')(x)
    x = tf.keras.layers.AveragePooling2D(2,2)(x)

    x = tf.keras.layers.Conv2D(64, kernel_size=3, activation = 'relu')(x)
    x = tf.keras.layers.Dropout(DROPOUT_FACTOR)(x)
    x = tf.keras.layers.AveragePooling2D(2,2)(x)

    return x

def build_model_adaptor(inputs):
    x = tf.keras.layers.Flatten()(inputs)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    return x

def build_classifier_head(inputs):
    return tf.keras.layers.Dense(CLASSES, activation='softmax', name = 'classifier_head')(inputs)

def build_regressor_head(inputs):
    return tf.keras.layers.Dense(units = '4', name = 'regressor_head')(inputs)

def build_model(inputs):

    feature_extractor = build_feature_extractor(inputs)

    model_adaptor = build_model_adaptor(feature_extractor)

    classification_head = build_classifier_head(model_adaptor)

    regressor_head = build_regressor_head(model_adaptor)

    model = tf.keras.Model(inputs = inputs, outputs = [classification_head, regressor_head])

    model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss = {'classifier_head' : 'categorical_crossentropy', 'regressor_head' : 'mse' },
              metrics = {'classifier_head' : 'accuracy', 'regressor_head' : 'mse' })

    return model

In [32]:
model = build_model(tf.keras.layers.Input(shape=(input_size, input_size, 1,)))

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 244, 244, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_3 (Conv2D)              (None, 242, 242, 16  160         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 average_pooling2d_3 (AveragePo  (None, 121, 121, 16  0          ['conv2d_3[0][0]']               
 oling2D)                       )                                                           

In [33]:
EPOCHS = 100

history = model.fit(train_ds,
                    steps_per_epoch=(len(training_files) // BATCH_SIZE),
                    validation_data=validation_ds, validation_steps=1,
                    epochs=EPOCHS)

Epoch 1/100
28/28 [==============================] - 3s 49ms/step - loss: 0.9247 - classifier_head_loss: 0.6794 - regressor_head_loss: 0.2453 - classifier_head_accuracy: 0.6083 - regressor_head_mse: 0.2453 - val_loss: 0.7096 - val_classifier_head_loss: 0.6725 - val_regressor_head_loss: 0.0371 - val_classifier_head_accuracy: 0.5781 - val_regressor_head_mse: 0.0371
Epoch 2/100
28/28 [==============================] - 1s 38ms/step - loss: 0.6533 - classifier_head_loss: 0.6258 - regressor_head_loss: 0.0275 - classifier_head_accuracy: 0.6283 - regressor_head_mse: 0.0275 - val_loss: 0.7016 - val_classifier_head_loss: 0.6809 - val_regressor_head_loss: 0.0207 - val_classifier_head_accuracy: 0.5781 - val_regressor_head_mse: 0.0207
Epoch 3/100
28/28 [==============================] - 1s 39ms/step - loss: 0.6277 - classifier_head_loss: 0.6039 - regressor_head_loss: 0.0237 - classifier_head_accuracy: 0.6306 - regressor_head_mse: 0.0237 - val_loss: 0.7047 - val_classifier_head_loss: 0.6895 - val_re

In [34]:
model.save('mask_detector.h5')